<a href="https://colab.research.google.com/github/KAUSHIK1224/DIY-projects/blob/main/emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Emotion Detection

In [ ]:
!pip install -q kaggle opencv-python tensorflow

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d deadskull7/fer2013
!unzip -q fer2013.zip -d /content/fer2013

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

IMAGE_SIZE = (48, 48)
BATCH_SIZE = 64

# Load data from CSV
df = pd.read_csv('/content/fer2013/fer2013.csv')

# Convert pixels to image data and labels
def preprocess_data(df):
    X = []
    y = []
    for index, row in df.iterrows():
        pixels = np.array(row['pixels'].split(), 'float32')
        img = pixels.reshape(48, 48)
        X.append(img)
        y.append(row['emotion'])
    X = np.array(X)
    X = np.expand_dims(X, -1) # Add channel dimension for grayscale
    y = tf.keras.utils.to_categorical(y, num_classes=7) # Assuming 7 emotion classes
    return X, y

X, y = preprocess_data(df)

# Split data into train and test sets (manual split as CSV doesn't have separate folders)
# Assuming 'Usage' column indicates train/test, if available. Otherwise, use train_test_split
if 'Usage' in df.columns:
    train_df = df[df['Usage'] == 'Training']
    test_df = df[df['Usage'] == 'PublicTest'] # Assuming PublicTest is the test set

    X_train, y_train = preprocess_data(train_df)
    X_test, y_test = preprocess_data(test_df)

    # Create ImageDataGenerators
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Use flow from numpy arrays as we have preprocessed the data
    train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
    test_generator = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE)

else:
    # If 'Usage' column is not available, use train_test_split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
    test_generator = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE)

print("Data generators created successfully.")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

NUM_CLASSES = 7

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

EPOCHS = 50

early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_emotion_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(
    train_generator,
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=X_test.shape[0] // BATCH_SIZE,
    callbacks=[early_stop, checkpoint]
)

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

def predict_emotion_cv(img_path, model, class_indices):
    # Load image with OpenCV
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    label_map = {v: k for k, v in class_indices.items()}
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48))
        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=(0, -1))
        prediction = model.predict(face_input)
        emotion_idx = np.argmax(prediction)
        emotion = label_map[emotion_idx]
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(img, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    # For Colab, use cv2_imshow
    from google.colab.patches import cv2_imshow
    cv2_imshow(img)

# Example usage:
# predict_emotion_cv('/content/fer2013/test/happiness/123.jpg', model, train_generator.class_indices)

In [ ]:
import matplotlib.pyplot as plt

loss, acc = model.evaluate(test_generator)
print(f"Test accuracy: {acc:.2f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('emotion_cnn_final.h5')
# To load later:
# from tensorflow.keras.models import load_model
# model = load_model('emotion_cnn_final.h5')

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np
import cv2
from tensorflow.keras.models import load_model

model = load_model('emotion_cnn_final.h5')
class_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def predict_emotion(image):
    # Convert to grayscale and resize
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(gray, (48, 48))
    face = face / 255.0
    face = np.expand_dims(face, axis=(0, -1))
    preds = model.predict(face)
    emotion = class_labels[np.argmax(preds)]
    return f"Predicted Emotion: {emotion.capitalize()}"

iface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Emotion Detection Dashboard",
    description="Upload a face image and the model will predict the emotion."
)

iface.launch(share=True)  # share=True gives you a public link

In [ ]:
!ls /content/fer2013